In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
raw_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [3]:
raw_df.createOrReplaceTempView("lending_club_data")

In [4]:
spark.sql("select * from lending_club_data")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,null,null,null,null,null,null,null

In [5]:
from pyspark.sql.functions import sha2,concat_ws

In [6]:
new_df = raw_df.withColumn("name_sha2", sha2(concat_ws("||", *["emp_title", "emp_length", "home_ownership", "annual_inc", "zip_code", "addr_state", "grade", "sub_grade","verification_status"]), 256))

In [7]:
new_df.createOrReplaceTempView("newtable")

In [8]:
spark.sql("select count(*) from newtable")

count(1)
2260701


In [9]:
spark.sql("select count(distinct(name_sha2)) from newtable")

count(DISTINCT name_sha2)
2257384


In [10]:
spark.sql("""select name_sha2, count(*) as total_cnt 
from newtable group by name_sha2 having total_cnt>1 order by total_cnt desc""")

name_sha2,total_cnt
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
f54295a60946dedad...,3
22593a1870543b2db...,3
5d52e7773cb0efff3...,3
059d401bb603d9a80...,3
819453be77718d747...,3


In [11]:
spark.sql("select * from newtable where name_sha2 like 'e3b0c44298fc1c149%'")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c

In [12]:
spark.sql("""select name_sha2 as member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,'USA' as country,grade,sub_grade,
verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint from newtable
""").repartition(1).write \
.option("header","true")\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/customers_data_csv") \
.save()

In [13]:
customers_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv006277/lendingclubproject/raw/customers_data_csv")

In [14]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
bfcec8da7b3f1fe83...,Regional Manager,8 years,RENT,130000.0,CT,069xx,USA,C,C3,Verified,135904.0,Individual,null,null
36399458db4295868...,null,null,RENT,12000.0,CA,953xx,USA,C,C1,Verified,7700.0,Joint App,62000.0,Verified
e4912b650878ca941...,Manager,6 years,MORTGAGE,103000.0,CA,922xx,USA,A,A2,Not Verified,511251.0,Individual,null,null
3d6f35ad2e2be4572...,Clinical Coordinator,6 years,MORTGAGE,64300.0,NM,871xx,USA,D,D1,Source Verified,233646.0,Individual,null,null
478e1cd37e35c20f7...,Warehouse supervisor,3 years,RENT,32000.0,NJ,073xx,USA,C,C4,Not Verified,44200.0,Individual,null,null
a5d42e35cc0be2789...,Sales Manager,5 years,MORTGAGE,140000.0,CA,917xx,USA,C,C5,Source Verified,455867.0,Individual,null,null
7f3ef55c784bf6b16...,Owner,10+ years,MORTGAGE,98000.0,NV,891xx,USA,A,A3,Source Verified,265000.0,Individual,null,null
beb30abaeca08f49b...,Alternate Media S...,2 years,RENT,56000.0,CA,940xx,USA,D,D1,Source Verified,75023.0,Individual,null,null
233e5e6797dd0a68b...,Hammerman,7 years,MORTGAGE,70000.0,OH,440xx,USA,C,C4,Source Verified,233341.0,Individual,null,null
c4b178ffaf80ed472...,Civil Engineer,< 1 year,RENT,120000.0,FL,321xx,USA,B,B3,Source Verified,97065.0,Individual,null,null


In [15]:
spark.sql("""select id as loan_id, name_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,
title from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/loans_data_csv") \
.save()

In [16]:
loans_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv006277/lendingclubproject/raw/loans_data_csv")

In [17]:
loans_df

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
141581221,9cb79aa7323e81be1...,24000.0,24000.0,60 months,25.34,709.23,Oct-2018,Current,small_business,Business
141506948,0dd2bbc517e3c8f9e...,10000.0,10000.0,36 months,7.84,312.63,Oct-2018,Current,credit_card,Credit card refin...
141357400,458458599d3df3bfc...,5600.0,5600.0,36 months,16.14,197.27,Oct-2018,Current,debt_consolidation,Debt consolidation
139445427,05ea141ec28b5c7f7...,9000.0,9000.0,36 months,8.46,283.95,Oct-2018,Current,debt_consolidation,Debt consolidation
141407409,aac68850fdac09fd0...,30000.0,30000.0,60 months,29.69,964.9,Oct-2018,Current,major_purchase,Major purchase
141360802,3a423e4589e89f429...,40000.0,40000.0,60 months,15.02,952.02,Oct-2018,Current,debt_consolidation,Debt consolidation
141163960,f1efcf7dfbfef21be...,40000.0,40000.0,36 months,12.73,1342.57,Oct-2018,Current,home_improvement,Home improvement
141533932,c89986155a070db2e...,10000.0,10000.0,60 months,14.47,235.13,Oct-2018,Current,debt_consolidation,Debt consolidation
141441276,118dc629b6e134419...,24000.0,24000.0,60 months,7.21,477.62,Oct-2018,Current,debt_consolidation,Debt consolidation
141569080,a86fa4b7493708333...,27525.0,27525.0,60 months,10.08,585.91,Oct-2018,Current,debt_consolidation,Debt consolidation


In [18]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/loans_repayments_csv") \
.save()

In [19]:
loans_repayments_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv006277/lendingclubproject/raw/loans_repayments_csv")

In [20]:
loans_repayments_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019
141360802,2313.98,2512.88,0.0,4826.86,952.02,Mar-2019,Apr-2019
141163960,4689.63,1994.93,0.0,6684.56,1342.57,Mar-2019,Apr-2019
141533932,585.29,640.53,15.0,1240.82,235.13,Mar-2019,Apr-2019
141441276,2030.82,762.81,0.0,2793.63,477.62,Mar-2019,Apr-2019
141569080,1803.55,1110.59,0.0,2914.14,585.91,Mar-2019,Apr-2019


In [21]:
spark.sql("""select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/loans_defaulters_csv") \
.save()

In [22]:
loans_defaulters_df = spark.read \
.format("csv") \
.option("InferSchema","true") \
.option("header","true") \
.load("/user/itv006277/lendingclubproject/raw/loans_defaulters_csv")

In [23]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
2209556c19f75a204...,0.0,0.0,1.0,1.0,0.0,0.0,null,65.0
9d0d53b8de898b446...,0.0,0.0,1.0,1.0,2.0,0.0,25.0,83.0
1a978870f4a9b4b43...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
c66ccc99ff4372809...,1.0,0.0,0.0,0.0,1.0,0.0,23.0,null
91d00a422248120a4...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
b7539a5579e723b21...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
2fca945a60ef9246c...,0.0,0.0,0.0,0.0,1.0,47.8,null,null
8a6dababd11fd592f...,0.0,0.0,0.0,0.0,2.0,0.0,null,null
3af72ce29e878edd7...,0.0,0.0,0.0,0.0,1.0,0.0,null,null
4395baca2ea2d58a8...,0.0,0.0,0.0,0.0,0.0,0.0,37.0,null
